# Projet Voltaire 2.0
## Préparation de la donnée 1 : sashimi de fichiers
### Equipe 242
---
Cet ensemble de fonctions sert comme premier parser de code en C : les fichiers sont découpés en tokens, ou mots, unité lexicale sur laquelle nous nous basons pour effectuer nos prédictions.

Voici toutes les fonctions avec un exemple d'utilisation pour chacune. Elles sont toutes localisées dans le fichier "file_sashimi.py"

1. _Variables globales_

Les delimiters representent l'ensemble des caractères de séparation (séparant les mots). Nous les avons stockés dans un tableau.

In [30]:
delimiters = [' ', '*', '/', ',', ';', '(', ')', '[', ']', '{', '}', '=', '<', '>', '+', '-', '&', '|', '!','?',':']


2. _Enlever les commentaires_

En C, 2 types de commentaires existent :
- ceux sur de multiples lignes (ici appeles c comments) commençant par `/*` et finissant par `*/`
- ceux sur une seule ligne (ici appeles cpp comments) commençant par `//`

Nous commençons par tous les enlever, vu qu'ils importent pas du tout

In [31]:
def remove_c_comments(st):
    while st.find('/*') != -1:
        if st.find('/*') != -1 and st.find('*/') != -1:
            a = st.find('/*')
            b = st.find('*/')
            if a < b:
                st = st[:a] + st[b + 2:]
            else:
                return "Erreur"
    return st

def remove_cpp_comment(st):
    while st.find('//') != -1:
        deb = st.find('//')
        fin = st[deb:].find('\n') + deb
        if fin != -1:
            st = st[:deb] + st[fin:]
        else:
            st = st[:deb]
    return st

print("Code de depart :\n\n==================")
code = "/* Voici ma tentative de coder \nune fonction calculant la moyenne en C */\n//test\nfloat average(float values[], int n_vals)\n{\nint i;\nfloat res=0;\nfor(i=0;i<n_vals;i++)\nres+=values[i];\nreturn (res/n_values);\nprintf(\"%c\", 'a');}"
print(code)
print("\n\nCode apres retrait des commentaires cpp :\n\n==================")
code = remove_cpp_comment(code)
print(code)
print("\n\nCode apres retrait des commentaires c :\n\n==================")
code = remove_c_comments(code)
print(code)

Code de depart :

/* Voici ma tentative de coder 
une fonction calculant la moyenne en C */
//test
float average(float values[], int n_vals)
{
int i;
float res=0;
for(i=0;i<n_vals;i++)
res+=values[i];
return (res/n_values);
printf("%c", 'a');}


Code apres retrait des commentaires cpp :

/* Voici ma tentative de coder 
une fonction calculant la moyenne en C */

float average(float values[], int n_vals)
{
int i;
float res=0;
for(i=0;i<n_vals;i++)
res+=values[i];
return (res/n_values);
printf("%c", 'a');}


Code apres retrait des commentaires c :



float average(float values[], int n_vals)
{
int i;
float res=0;
for(i=0;i<n_vals;i++)
res+=values[i];
return (res/n_values);
printf("%c", 'a');}


3. _Enlèvement des caractères d'échappement_

Contrairement a du python, l'indentation et les retours a la ligne n'importent pas en C. Nous les avons donc retirés

In [32]:
def remove_char(st, c):
    return "".join([i for i in st if i != c])

print("Code a la sortie precedente :\n\n==================")
print(code)
code = remove_char(code, '\t')
code = remove_char(code, '\n')
code = remove_char(code, '\r')
print("\n\nCode apres retrait des retours a la lignes, tabulation et retour chariot :\n\n==================\n")
print(code)

Code a la sortie precedente :



float average(float values[], int n_vals)
{
int i;
float res=0;
for(i=0;i<n_vals;i++)
res+=values[i];
return (res/n_values);
printf("%c", 'a');}


Code apres retrait des retours a la lignes, tabulation et retour chariot :


float average(float values[], int n_vals){int i;float res=0;for(i=0;i<n_vals;i++)res+=values[i];return (res/n_values);printf("%c", 'a');}


3. _Découpage des chaines de caractere/caractère_

Il est impératif de traiter les string/caracteres, mis entre `"` ou `'` comme un cas a part ! Imaginons une seconde qu'il y ait du code entre les guillemets, on ne voudrait pas le considerer comme tel ! Il nous a donc semblé logique de commencer le découpage par la.

In [33]:
def remove_blank_table(t) :
    ta = []
    for it in t :
        if len(it) != 0 :
            ta.append(it)
    return ta

def sashimi_string(st):
    t = []
    while st.find('"') != -1:
        if st[st.find('"'):].find('"') != -1 :
            t.append(st[:st.find('"')])
            t.append(st[st.find('"'): st[st.find('"')+1:].find('"')+st.find('"')+2])
            st = st[st[st.find('"')+1:].find('"')+st.find('"')+2:]
        else :
            st = st[st.find('"')+1:]
    t.append(st)
    return remove_blank_table(t)

def sashimi_ch(ta):
    t = []
    for st in ta :
        if st[0] != '"' :
            while st.find("'") != -1:
                if st[st.find("'"):].find("'") != -1 :
                    t.append(st[:st.find("'")])
                    t.append(st[st.find("'"): st[st.find("'")+1:].find("'")+st.find("'")+2])
                    st = st[st[st.find("'")+1:].find("'")+st.find("'")+2:]
                else :
                    st = st[st.find("'")+1:]
            t.append(st)
        else :
            t.append(st)
    return remove_blank_table(t)

code = sashimi_string(code)
code = sashimi_ch(code)
print("\n\nCode apres decoupages des strings/caracteres :\n\n==================")
print(code)



Code apres decoupages des strings/caracteres :

['float average(float values[], int n_vals){int i;float res=0;for(i=0;i<n_vals;i++)res+=values[i];return (res/n_values);printf(', '"%c"', ', ', "'a'", ');}']


4. _C'est l'heure du sashimi !_

Le code ainsi traité est prêt a être découpé en mots ! A chaque caractere de séparation (les delimiters vus plus tot), on effectue un tranchage. Parmi les delimiters, un n'est pas utile semantiquement parlant : l'espace. Pour eviter des calculs inutiles, on le retire

In [34]:
def remove_space(t):
    return [i for i in t if i!=' ']

def sashimi_char(st, c):
    tab = []
    it = 0
    a = 0
    if st == '':
        return []
    if st[0] == '"' :
        return [st]
    while a < len(st) and st[a] == c:
        it += 1
        tab.append(c)
        a += 1
    if a < len(st):
        tab.append(st[a])
    a += 1
    for i in st[a:]:
        if i == c:
            it += 2
            tab.append(c)
            if a != len(st) - 1:
                tab.append('')
        else:
            tab[it] = tab[it] + i
        a += 1
    return tab

print("\n\nCode apres sashimi complet :\n\n==================")
for deli in delimiters:
    t = []
    for troncon in code:
        if troncon != deli:
            t += sashimi_char(troncon, deli)
        else:
            t += deli
    code = t
code = remove_space(code)
print(code)



Code apres sashimi complet :

['float', 'average', '(', 'float', 'values', '[', ']', ',', 'int', 'n_vals', ')', '{', 'int', 'i', ';', 'float', 'res', '=', '0', ';', 'for', '(', 'i', '=', '0', ';', 'i', '<', 'n_vals', ';', 'i', '+', '+', ')', 'res', '+', '=', 'values', '[', 'i', ']', ';', 'return', '(', 'res', '/', 'n_values', ')', ';', 'printf', '(', '"%c"', ',', "'a'", ')', ';', '}']


Notre code ainsi finement découpé est prêt a être transformé !